# Get The Coins Camera Calibration

## 1. Needed Libraries

In [ ]:
# External libraries

import cv2
import numpy as np
import glob
import os
from time import sleep

# Own libraries

from my_logger import MyLogger

## 2. Create logger instance

In [ ]:
logger = MyLogger(level='DEBUG', show_timestamp=False)

## 3. Capture images for calibration

It is necessary to capture at least 4 images.

In [ ]:
# Define the output directory for image capturing
output_dir = "./media/calibration/calibration_images_test"

# Open device
video = cv2.VideoCapture(2)

if video.isOpened():

    # STEP 1. Create window for displaying -----------------------------

    window_name = f'Showing video stream'
    # Opens a window to show the video stream
    cv2.namedWindow(window_name)

    # STEP 2. Definition of control variables --------------------------

    window_is_visible = True
    key = -1  # to start key must be different to ord('q') = 113
    frame_count = 0  # index to know the current frame
    key_wait_delay = 1  # delay time in milliseconds

    # STEP 3. Create the output directory if it does not exist ---------

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # STEP 4. Start frames displaying ----------------------------------

    logger.info('Start of video streaming.')
    logger.info("Press 'c' for image capturing.")
    logger.info("Press 'q' to end image capture.")

    while ((video.isOpened()) and (key != ord('q'))
            and window_is_visible):

        ret_ok, frame = video.read()

        if ret_ok:
            if (key == ord('c')):
                path = os.path.join(
                    output_dir, f'{frame_count:05d}.bmp')

                cv2.imwrite(path, frame)
                logger.debug(f'Saved {path}')

            frame_count += 1

            # Defining the dimensions of checkerboard
            pattern_size = (6, 9)

            # Find the chess board corners
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # If desired number of corners are found in the image
            # then ret = true
            ret, corners = cv2.findChessboardCorners(
                gray, pattern_size, None)

            frame = cv2.drawChessboardCorners(
                frame, pattern_size, corners, ret)

            cv2.imshow(window_name, frame)

            # Press 'q' key to finish the while loop
            key = cv2.waitKey(key_wait_delay)

            # Check if the window has been closed
            if cv2.getWindowProperty(
                window_name, cv2.WND_PROP_VISIBLE) == 0:
                window_is_visible = False
        else:
            break

    logger.info('End of video streaming.')

    cv2.destroyAllWindows()

else:
    logger.error('To display a stream you must first configure a '
                 'correct video streaming address.')

## 4. Calibration of intrinsic camera parameters

In [ ]:
# Chessboard dimensions
number_of_squares_X = 7  # Squares along the X-axis
number_of_squares_Y = 10  # Squares along the Y-axis
nX = number_of_squares_X - 1  # Interior corners along the X-axis
nY = number_of_squares_Y - 1  # Interior corners along the Y-axis
square_size = 0.026  # Size of a square in meters
  
# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 12000, 0.0000001)

# Define the 3D coordinates of the points in the world coordinate frame
object_points_3D = np.zeros((nX * nY, 3), np.float32)  
object_points_3D[:,:2] = np.mgrid[0:nY, 0:nX].T.reshape(-1, 2) 
object_points_3D = object_points_3D * square_size

# Lists to store object and image points from all images
object_points = []  # 3D points in world space
image_points = []   # 2D points in image plane

# Get the file path for images in the current directory
images = glob.glob('./media/calibration/calibration_images_test/*.bmp')

for image_file in images:
    image = cv2.imread(image_file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect chessboard corners
    success, corners = cv2.findChessboardCorners(gray, (nY, nX), None)

    if success:
        object_points.append(object_points_3D)

        # Refine corner accuracy
        corners_2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        image_points.append(corners_2)

        # Draw and show corners
        cv2.drawChessboardCorners(image, (nY, nX), corners_2, success)
        cv2.imshow("Image", image)
        cv2.waitKey(100)
                                                                                                                    
# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
  object_points, image_points, gray.shape[::-1], None, None
)

# Save calibration results to file
cv_file = cv2.FileStorage('./media/calibration/intrinsec_parameters_test.yaml', cv2.FILE_STORAGE_WRITE)
cv_file.write('K', mtx)  # Save the camera matrix
cv_file.write('D', dist)  # Save the distortion coefficients
cv_file.release()
  
# Display the camera calibration results
logger.info("Camera matrix:") 
logger.info(f"\n{mtx}") 

logger.info("Distortion coefficients:") 
logger.info(f"\n{dist}") 

# Close all open windows
cv2.destroyAllWindows()

## 4. Create ArUco markers

In [ ]:
# create the dictionary for markers type
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
size_of_marker = 400  # size of marker.

# generating IDs with for loop
for marker_id in range(2):
    # generating the marker
    img = cv2.aruco.generateImageMarker(dictionary, marker_id, size_of_marker)

    print("Dimension of Marker: ", img.shape)
    # save/write the image
    cv2.imwrite("./markers/DICT_6X6_250_{}.png".format(marker_id), img)

# display the image(marker) on windows
cv2.imshow("Marker", img)
cv2.waitKey(0)
cv2.destroyAllWindows()